In [5]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import re

import warnings
warnings.filterwarnings('ignore')

import piccard as pc
import piccard2 as pc2

from tscluster.opttscluster import OptTSCluster
# from tscluster.tskmeans import TSKmeans, TSGlobalKmeans
from tscluster.preprocessing.utils import load_data
# from tscluster.metrics import inertia, max_dist
# from tscluster.tsplot import tsplot

In [2]:
ct_2006 = gpd.read_file("data_testing/gct_000b06a_e/gct_000b06a_e.shp")
ct_2011 = gpd.read_file("data_testing/gct_000b11a_e/gct_000b11a_e.shp")
ct_2016 = gpd.read_file("data_testing/lct_000b16a_e/lct_000b16a_e.shp")
ct_2021 = gpd.read_file("data_testing/lct_000b21a_e/lct_000b21a_e.shp")

In [ ]:
census_dfs = [ct_2011, ct_2016, ct_2021]
years = ['2011', '2016', '2021']

network_table = pc.create_network_table(census_dfs, years, 'CTUID')

In [4]:
network_table

,ctuid_2011,ctuid_2016,ctuid_2021,ctname_2011,cmauid_2011,cmaname_2011,cmatype_2011,cmapuid_2011,pruid_2011,prname_2011,...,cmatype_2016,cmapuid_2016,pruid_2016,prname_2016,network_level_2016,ctname_2021,pruid_2021,dguid_2021,landarea_2021,network_level_2021
0,NaN,2016_3050120.00,2021_3050120.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,B,13305,13,New Brunswick / Nouveau-Brunswick,2,0120.00,13,2021S05073050120.00,149.1445,3
1,NaN,2016_3100140.00,2021_3100140.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,B,13310,13,New Brunswick / Nouveau-Brunswick,2,0140.00,13,2021S05073100140.00,144.7702,3
2,NaN,2016_3200028.00,2021_3200028.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,K,13320,13,New Brunswick / Nouveau-Brunswick,2,0028.00,13,2021S05073200028.00,29.4354,3
3,NaN,2016_3200029.00,2021_3200029.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,K,13320,13,New Brunswick / Nouveau-Brunswick,2,0029.00,13,2021S05073200029.00,527.2040,3
4,NaN,2016_3200030.00,2021_3200030.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,K,13320,13,New Brunswick / Nouveau-Brunswick,2,0030.00,13,2021S05073200030.00,294.3542,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,2011_9700100.00,2016_9700100.00,2021_9700100.00,0100.00,970,Prince George,K,59970,59,British Columbia / Colombie-Britannique,...,K,59970,59,British Columbia / Colombie-Britannique,2,0100.00,59,2021S05079700100.00,12487.2890,3
6340,2011_9700101.00,2016_9700101.00,2021_9700101.00,0101.00,970,Prince George,K,59970,59,British Columbia / Colombie-Britannique,...,K,59970,59,British Columbia / Colombie-Britannique,2,0101.00,59,2021S05079700101.00,667.6134,3
6341,2011_9700102.00,2016_9700102.00,2021_9700102.00,0102.00,970,Prince George,K,59970,59,British Columbia / Colombie-Britannique,...,K,59970,59,British Columbia / Colombie-Britannique,2,0102.00,59,2021S05079700102.00,2802.3279,3
6342,2011_9700103.00,2016_9700103.01,2021_9700103.01,0103.00,970,Prince George,K,59970,59,British Columbia / Colombie-Britannique,...,K,59970,59,British Columbia / Colombie-Britannique,2,0103.01,59,2021S05079700103.01,5.2340,3


In [ ]:
pc2.clustering_prep(network_table, 'ctuid')

In [20]:
households_data_2021 = gpd.read_file("data_testing/households_data_2021.geojson")
households_data_2016 = gpd.read_file("data_testing/households_data_2016.geojson")
households_data_2011 = gpd.read_file("data_testing/households_data_2011.geojson")

households_data_2021.head(5)

,CSD_UID,CMA_UID,name,Dwellings 2016,Population,Dwellings,Population 2016,Households,Type,GeoUID,Households 2016,Quality Flags,Shape Area,CD_UID,Region Name,Area (sq km),v_CA21_434: Occupied private dwellings by structural type of dwelling data,v_CA21_435: Single-detached house,v_CA21_440: Apartment in a building that has five or more storeys,geometry
0,3520005,35535,0001.00,274,599,253,595,235,CT,5350001.00,247,0,6.8192,3520,0001.00,6.8192,235.0,15.0,0.0,"MULTIPOLYGON (((-79.33526 43.62681, -79.33561 ..."
1,3520005,35535,0002.00,279,604,294,620,284,CT,5350002.00,270,0,3.3926,3520,0002.00,3.3926,285.0,250.0,0.0,"MULTIPOLYGON (((-79.38245 43.62556, -79.382 43..."
2,3520005,35535,0003.00,267,457,279,749,265,CT,5350003.00,258,0,0.9455,3520,0003.00,0.9455,265.0,0.0,265.0,"MULTIPOLYGON (((-79.43466 43.63369, -79.43328 ..."
3,3520005,35535,0004.00,3609,6306,3620,6686,3276,CT,5350004.00,3455,0,0.3404,3520,0004.00,0.3404,3275.0,70.0,1790.0,"MULTIPOLYGON (((-79.43466 43.63369, -79.43566 ..."
4,3520005,35535,0005.00,3729,6957,4235,6424,3720,CT,5350005.00,3424,0,0.3764,3520,0005.00,0.3764,3720.0,40.0,2420.0,"MULTIPOLYGON (((-79.43603 43.63717, -79.43726 ..."


In [22]:
households_data_2016.head(5)

,CSD_UID,CMA_UID,name,Dwellings 2016,Population,Dwellings,Population 2016,Households,Type,GeoUID,Households 2016,Quality Flags,Shape Area,CD_UID,Region Name,Area (sq km),v_CA16_408: Occupied private dwellings by structural type of dwelling data,v_CA16_409: Single-detached house,v_CA16_410: Apartment in a building that has five or more storeys,geometry
0,3520005,35535,0001.00,274,599,253,595,235,CT,5350001.00,247,0,6.8192,3520,0001.00,6.8192,250.0,15.0,0.0,"MULTIPOLYGON (((-79.33526 43.62681, -79.33561 ..."
1,3520005,35535,0002.00,279,604,294,620,284,CT,5350002.00,270,0,3.3926,3520,0002.00,3.3926,270.0,245.0,0.0,"MULTIPOLYGON (((-79.38245 43.62556, -79.382 43..."
2,3520005,35535,0003.00,267,457,279,749,265,CT,5350003.00,258,0,0.9455,3520,0003.00,0.9455,260.0,0.0,255.0,"MULTIPOLYGON (((-79.43466 43.63369, -79.43328 ..."
3,3520005,35535,0004.00,3609,6306,3620,6686,3276,CT,5350004.00,3455,0,0.3404,3520,0004.00,0.3404,3455.0,70.0,1885.0,"MULTIPOLYGON (((-79.43466 43.63369, -79.43566 ..."
4,3520005,35535,0005.00,3729,6957,4235,6424,3720,CT,5350005.00,3424,0,0.3764,3520,0005.00,0.3764,3420.0,40.0,2085.0,"MULTIPOLYGON (((-79.43603 43.63717, -79.43726 ..."


In [23]:
households_data_2011.head(5)

,Shape Area,Quality Flags,GeoUID,Type,Households,PR_UID,Dwellings,NHS Non-Return Rate,Population,CMA_UID,CSD_UID,CD_UID,Region Name,Area (sq km),v_CA11F_199: Total number of occupied private dwellings by structural type of dwelling,v_CA11F_200: Single-detached house,"v_CA11F_201: Apartment, building that has five or more storeys",geometry
0,6.62211,00000,5350001.00,CT,264,35,283,0.368,604,35535,3520005,3520,Toronto,6.62211,265.0,10.0,0.0,"MULTIPOLYGON (((-79.30394 43.66433, -79.30409 ..."
1,3.27377,00000,5350002.00,CT,290,35,296,0.307,657,35535,3520005,3520,Toronto,3.27377,290.0,270.0,5.0,"MULTIPOLYGON (((-79.36966 43.62278, -79.36968 ..."
2,0.94253,02000,5350003.00,CT,227,35,262,NaN,673,35535,3520005,3520,Toronto,0.94253,225.0,0.0,225.0,"MULTIPOLYGON (((-79.45379 43.63922, -79.45172 ..."
3,0.34347,01000,5350004.00,CT,3416,35,3634,0.391,6781,35535,3520005,3520,Toronto,0.34347,3415.0,65.0,1915.0,"MULTIPOLYGON (((-79.43603 43.63717, -79.43466 ..."
4,0.37612,01000,5350005.00,CT,2782,35,3472,0.321,5510,35535,3520005,3520,Toronto,0.37612,2785.0,45.0,1435.0,"MULTIPOLYGON (((-79.42141 43.64066, -79.42132 ..."
